<a href="https://colab.research.google.com/github/hadwin-357/ProteinMPNN_breakdown/blob/main/model_utils_function_8_Encoder_Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#test model utils function
#Encoder - decoder
Encoder:
 1.take h_E (edge feature) (postional + geometry) and h_V(initialize with zero -1 dimension is the same as h_E)\
 2.concat: neighbor nodes embedding and edge embedding to h_EV  [B,N,K,C+C]\
3.concat: h_V with h_EV (center nodes + neigher nodes
and edges) [B,N,K,C+C+C]
4. MLP: three layers of MLP and Activation to generate h_message [B,N,K, num_hidden]
5. updata h_message with attention_mask if there is one   [B,N,K,num_hidden]
5. sum over the K dimension and scaled by a scaling factor -> dh [B,N,num_hidden]
6. norm(dropout(dh) + h_V) --> h_V (as a residual ?)
7. MLP(dh) ->dh  [B,N,num_hidden]
8. norm(h_V +dropout(dh)) --> udpate h_V  [B, N, num_hidden]
9. add mask_V if there is one -> h_V


---



The high-level summary: collect neighbor embedding, combine self embeddeing, Go through MLP to form message;
The following is how the message is used to update the h_V node embedding\
1.sum over message by -2 dimension and add the result to h_V
2.MLP through the summarized message and add the reuslt to h_V again

similarly, the h_E will be updated through the message.
In each layer, the update h_V and h_E will be used to update each other in a evolving way

---



Decoder:
Similar to Encoder, but only h_V will be updated
the upated h_V will be used for prediction



In [3]:
import torch
import torch.nn as nn
class EncLayer(nn.Module):
    def __init__(self, num_hidden, num_in, dropout=0.1, num_heads=None, scale=30):
        super(EncLayer, self).__init__()
        self.num_hidden = num_hidden
        self.num_in = num_in
        self.scale = scale
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(num_hidden)
        self.norm2 = nn.LayerNorm(num_hidden)
        self.norm3 = nn.LayerNorm(num_hidden)

        self.W1 = nn.Linear(num_hidden + num_in, num_hidden, bias=True)
        self.W2 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.W3 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.W11 = nn.Linear(num_hidden + num_in, num_hidden, bias=True)
        self.W12 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.W13 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.act = torch.nn.GELU()
        self.dense = PositionWiseFeedForward(num_hidden, num_hidden * 4)

    def forward(self, h_V, h_E, E_idx, mask_V=None, mask_attend=None):
        """ Parallel computation of full transformer layer """

        h_EV = cat_neighbors_nodes(h_V, h_E, E_idx)
        h_V_expand = h_V.unsqueeze(-2).expand(-1,-1,h_EV.size(-2),-1)
        h_EV = torch.cat([h_V_expand, h_EV], -1)
        h_message = self.W3(self.act(self.W2(self.act(self.W1(h_EV)))))
        if mask_attend is not None:
            h_message = mask_attend.unsqueeze(-1) * h_message
        dh = torch.sum(h_message, -2) / self.scale
        h_V = self.norm1(h_V + self.dropout1(dh))

        dh = self.dense(h_V)
        h_V = self.norm2(h_V + self.dropout2(dh))
        if mask_V is not None:
            mask_V = mask_V.unsqueeze(-1)
            h_V = mask_V * h_V

        h_EV = cat_neighbors_nodes(h_V, h_E, E_idx)
        h_V_expand = h_V.unsqueeze(-2).expand(-1,-1,h_EV.size(-2),-1)
        h_EV = torch.cat([h_V_expand, h_EV], -1)
        h_message = self.W13(self.act(self.W12(self.act(self.W11(h_EV)))))
        h_E = self.norm3(h_E + self.dropout3(h_message))
        return h_V, h_E

In [4]:
def cat_neighbors_nodes(h_nodes, h_neighbors, E_idx):
    h_nodes = gather_nodes(h_nodes, E_idx)  # B, N, K, C
    h_nn = torch.cat([h_neighbors, h_nodes], -1)
    return h_nn

In [ ]:
'''
# h_V: node embedding [B,N,C]h_E edge_embedding [B,N,K,C], E_idx: neighbor idx

h_EV = cat_neighbors_nodes(h_V, h_E, E_idx) # combine neighbor node and edge embedding
h_V_expand = h_V.unsqueeze(-2).expand(-1,-1,h_EV.size(-2),-1)  [B,N,k,C]
h_EV = torch.cat([h_V_expand, h_EV], -1) # add self node embedding
h_message = self.W3(self.act(self.W2(self.act(self.W1(h_EV))))) # forming message
if mask_attend is not None:
    h_message = mask_attend.unsqueeze(-1) * h_message
dh = torch.sum(h_message, -2) / self.scale # sum over -2 dimension (neighbors)
h_V = self.norm1(h_V + self.dropout1(dh)) # use summarized message to update h_V

dh = self.dense(h_V)
h_V = self.norm2(h_V + self.dropout2(dh))
if mask_V is not None:
    mask_V = mask_V.unsqueeze(-1)
    h_V = mask_V * h_V

h_EV = cat_neighbors_nodes(h_V, h_E, E_idx)
h_V_expand = h_V.unsqueeze(-2).expand(-1,-1,h_EV.size(-2),-1)
h_EV = torch.cat([h_V_expand, h_EV], -1)
h_message = self.W13(self.act(self.W12(self.act(self.W11(h_EV)))))
h_E = self.norm3(h_E + self.dropout3(h_message)) # use summarized message to update h_E
'''

#Decoder

In [5]:
class DecLayer(nn.Module):
    def __init__(self, num_hidden, num_in, dropout=0.1, num_heads=None, scale=30):
        super(DecLayer, self).__init__()
        self.num_hidden = num_hidden
        self.num_in = num_in
        self.scale = scale
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(num_hidden)
        self.norm2 = nn.LayerNorm(num_hidden)

        self.W1 = nn.Linear(num_hidden + num_in, num_hidden, bias=True)
        self.W2 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.W3 = nn.Linear(num_hidden, num_hidden, bias=True)
        self.act = torch.nn.GELU()
        self.dense = PositionWiseFeedForward(num_hidden, num_hidden * 4)

    def forward(self, h_V, h_E, mask_V=None, mask_attend=None):
        """ Parallel computation of full transformer layer """

        # Concatenate h_V_i to h_E_ij
        h_V_expand = h_V.unsqueeze(-2).expand(-1,-1,h_E.size(-2),-1)
        h_EV = torch.cat([h_V_expand, h_E], -1)

        h_message = self.W3(self.act(self.W2(self.act(self.W1(h_EV)))))
        if mask_attend is not None:
            h_message = mask_attend.unsqueeze(-1) * h_message
        dh = torch.sum(h_message, -2) / self.scale

        h_V = self.norm1(h_V + self.dropout1(dh))

        # Position-wise feedforward
        dh = self.dense(h_V)
        h_V = self.norm2(h_V + self.dropout2(dh))

        if mask_V is not None:
            mask_V = mask_V.unsqueeze(-1)
            h_V = mask_V * h_V
        return h_V

In [ ]:
'''
h_V_expand = h_V.unsqueeze(-2).expand(-1,-1,h_E.size(-2),-1)  # expand [B,N,C] -> [B,N,K,C]
h_EV = torch.cat([h_V_expand, h_E], -1) # add edge embeddign to h_V

h_message = self.W3(self.act(self.W2(self.act(self.W1(h_EV))))) # form message
if mask_attend is not None:
    h_message = mask_attend.unsqueeze(-1) * h_message
dh = torch.sum(h_message, -2) / self.scale  # sum message over neighbors

h_V = self.norm1(h_V + self.dropout1(dh)) # used summarized message to update h_V

# Position-wise feedforward
dh = self.dense(h_V)
h_V = self.norm2(h_V + self.dropout2(dh))

if mask_V is not None:
    mask_V = mask_V.unsqueeze(-1)
    h_V = mask_V * h_V
return h_V
'''